<a href="https://colab.research.google.com/github/ayeshasGithub/hello-world/blob/master/Read_plc_tags.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Kafka depedencies

In [ ]:
#new 3
import json 
from kafka import KafkaConsumer
import redis
import time
import sys

topic_name = 'magnaPlcTagData'
b_server=['10.0.1.96:9092']


#redis host
redis_host="192.168.2.11"
redis_port = 6379
redis_password = ""
r=redis.Redis(host="192.168.2.11", port=6379, db=0)

#key conversion: {'line': 'rw43', 'zone': 'zone01', 'station': '-1', 'asset': '-1', 'tag_name': 'Zone01_Shift_1', 
#                   'tag_value': '1', 'tag_datetime':'2020-05-28T11:31:05.89'} 
key_conversion={'root.LaneID':'line','root.ZoneID':'zone','root.TagID':'tag_name','root.StationID':'station',
                
                        'root.EquipmentID':'asset','root.TagValue':'tag_value','root.TagDateTime':'tag_datetime'}

msg_count=0
#plc_tag_hash={}#dic of dic
plc_tag_msg={}

consumer = KafkaConsumer(topic_name, bootstrap_servers=b_server,auto_offset_reset='earliest',
                         session_timeout_ms=10000, 
                          enable_auto_commit=False, auto_commit_interval_ms=5000,
                         reconnect_backoff_ms=30000, reconnect_backoff_max_ms=60000,
                         max_poll_records=50, max_poll_interval_ms=300000,                        
                         group_id=None, value_deserializer=lambda x: loads(x.decode('utf-8')))


for msg in consumer:
    msg_count=msg_count+1
    #if msg_count<= 30000:
    #    continue
    msg = msg.value
    
    #print('{}'.format(msg))
  
    #parse tag from msg
    dic_msg=msg[0]#type(msg)=list, type(msg[0])=dic\n",
    res=json.loads(dic_msg['FeedData'])#type(dic_msg['FeedData'])=str, type(dic_msg['FeedData'][0])=type(res)=list\n",
    new_dic=res[0]#type(new_dic)=dic\n",
    name_list=new_dic['NAMES']#type(new_dic['NAMES'])=list\n",
    
    len_value=len(new_dic['VALUES'])
    print('number of values',len_value)
    
    for i in range(len_value):
        
        value_list=new_dic['VALUES'][i]#type(new_dic['Values'])=list of list\n",
        #value_list=new_dic['VALUES'][0]#type(new_dic['Values'])=list of list\n",
    
         
        #make one dic from one parsed msg => tag
        plc_dic={}
        for name,value in zip(name_list,value_list):
            #print(name,':',value)
            if value is None:
                value=str(-1)
            plc_dic[key_conversion[name]]=str(value).lower()
        
        #print(plc_dic)
    
        #make primary key for the tag
        pk_tag=plc_dic['line']+'.'+plc_dic['zone']+'.'+plc_dic['station']+'.'+plc_dic['asset']+'.'+plc_dic['tag_name']
        pk_tag=pk_tag.replace('.-1','')
        #print(pk_tag)
    
        plc_tag_msg[pk_tag]=plc_dic
    
    
        #redis write    
        r_val=json.dumps(plc_tag_msg)
        #r.set('plc_tag_hash',r_val)#UNCOMMENT
        #r.hset('plc_tag_experiment',pk_tag,json.dumps(plc_dic))#UNCOMMENT
    
        #redis read
        '''
        r_had=r.get('plc_tag_hash')
        plc_tag=json.loads(r_had)
        #print('from redis read: ',plc_tag)
        #'''    
    
    
    
        #print(plc_dic)
        print('tag_name: ',plc_dic['tag_name'])
        #print('tag_value: ',plc_dic['tag_value'])
    
    if msg_count%500==0:
        print(msg_count,end=' ')
    if msg_count==3:#0001:
        print('done')
        break
    #break

In [ ]:
#redis experiment

#initialize dic
new_dic={}

new_dic[0]='we'
new_dic['1']='he'

print(new_dic)


#redis host
redis_host="192.168.2.11"
redis_port = 6379
redis_password = ""
r=redis.StrictRedis(host="192.168.2.11", port=6379, db=0)
#'''


key='any'
#redis write    
#r_val=json.dumps(plc_tag_msg)
r.hset('plc_tag_experiment',key,json.dumps(new_dic))#UNCOMMENT
    
#redis read
r_haddy=r.hgetall('plc_tag_experiment')
print('from redis read: ',r_haddy)
#'''    
    

In [ ]:
#Experiment kafka
import json 
from kafka import KafkaConsumer
import redis
import time
import sys

topic_name = 'magnaPlcTagData'
b_server=['10.0.1.96:9092']


msg_count=0

consumer = KafkaConsumer(topic_name, bootstrap_servers=b_server,auto_offset_reset='latest',
                         session_timeout_ms=10000, 
                          enable_auto_commit=False, auto_commit_interval_ms=5000,
                         reconnect_backoff_ms=30000, reconnect_backoff_max_ms=60000,
                         max_poll_records=50, max_poll_interval_ms=300000,
                         
                         group_id=None, value_deserializer=lambda x: loads(x.decode('utf-8')))
for msg in consumer:
    msg_count=msg_count+1
    msg = msg.value
    
    if msg_count==5:
        print(msg_count)
        break

In [ ]:
#Redis
import the Redis client
import redis


# Create a redis client
redisClient = redis.StrictRedis(host='localhost', port=6379, db=0)

# Add key value pairs to the Redis hash
hashName = "Dessert"
redisClient.hset(hashName, 'line', "Cheesecake")
redisClient.hset(hashName, 'zone', "zone01")
redisClient.hset(hashName, 'station', "-1")
redisClient.hset(hashName, 'asset', "-1")
redisClient.hset(hashName, 'tag_name', "Zone01_Shift_1")
redisClient.hset(hashName, 'tag_value', "1")

# Print the hash
print(redisClient.hgetall(hashName))pl

# Remove a key
redisClient.hdel(hashName, 1)
 

# Print the hash after removing a key
print(redisClient.hgetall(hashName))

{'line': 'rw43',
 'zone': 'zone01',
 'station': '-1',
 'asset': '-1',
 'tag_name': 'Zone01_Shift_1',
 'tag_value': '1'
} 



In [ ]:
#new 1
import json 
from kafka import KafkaConsumer
import redis
import time
import sys

topic_name = 'magnaPlcTagData'
b_server=['10.0.1.96:9092']


#redis host
redis_host="192.168.2.11"
redis_port = 6379
redis_password = ""
r=redis.Redis(host="192.168.2.11", port=6379, db=0)

#key conversion: {'line': 'rw43', 'zone': 'zone01', 'station': '-1', 'asset': '-1', 'tag_name': 'Zone01_Shift_1', 
#                   'tag_value': '1', 'tag_datetime':'2020-05-28T11:31:05.89'} 
key_conversion={'root.LaneID':'line','root.ZoneID':'zone','root.TagID':'tag_name','root.StationID':'station',
                
                        'root.EquipmentID':'asset','root.TagValue':'tag_value','root.TagDateTime':'tag_datetime'}

msg_count=0
plc_tag_hash={}#dic of dic
plc_tag_msg={}
consumer = KafkaConsumer(topic_name, bootstrap_servers=b_server,auto_offset_reset='earliest',
                          enable_auto_commit=True, value_deserializer=lambda x: loads(x.decode('utf-8')))
for msg in consumer:
    msg_count=msg_count+1
    msg = msg.value
    
    #print('\n')
    sys.stdout.write('%d'%msg_count)
    #print(msg_count)
    sys.stdout.flush()
    #time.sleep(0.1)
    
    #print('{}'.format(msg))
  
    #parse tag from msg
    dic_msg=msg[0]#type(msg)=list, type(msg[0])=dic\n",
    res=json.loads(dic_msg['FeedData'])#type(dic_msg['FeedData'])=str, type(dic_msg['FeedData'][0])=type(res)=list\n",
    new_dic=res[0]#type(new_dic)=dic\n",
    name_list=new_dic['NAMES']#type(new_dic['NAMES'])=list\n",
    value_list=new_dic['VALUES'][0]#type(new_dic['Values'])=list of list\n",
  
    #make one dic from one parsed msg => tag
    plc_dic={}
    for name,value in zip(name_list,value_list):
        #print(name,':',value)
        if value is None:
            value=str(-1)
        plc_dic[key_conversion[name]]=str(value).lower()
        
    #print(plc_dic)
    
    #make primary key for the tag
    pk_tag=plc_dic['line']+'.'+plc_dic['zone']+'.'+plc_dic['station']+'.'+plc_dic['asset']+'.'+plc_dic['tag_name']
    pk_tag=pk_tag.replace('.-1','')
    #print(pk_tag)
    
    plc_tag_msg[pk_tag]=plc_dic
    
    
    #redis write    
    r_val=json.dumps(plc_tag_msg)
    r.set('plc_tag_hash',r_val)
    
    #redis read
    r_had=r.get('plc_tag_hash')
    plc_tag=json.loads(r_had)
    #print('from redis read: ',plc_tag)
    #'''    
    
    
    
    #print(plc_dic)
    #print('tag_name: ',plc_dic['tag_name'])
    #print('tag_value: ',plc_dic['tag_value'])
    
    
    if msg_count%100==0:
        print(msg_count,end=' ')
    if msg_count==30000:
        print('done')
        break
    #break

ModuleNotFoundError: ignored

In [ ]:
#new
import json 
from kafka import KafkaConsumer
import redis
import time
import sys

topic_name = 'magnaPlcTagData'
b_server=['10.0.1.96:9092']


#redis host
redis_host="192.168.2.11"
redis_port = 6379
redis_password = ""
r=redis.Redis(host="192.168.2.11", port=6379, db=0)

#key conversion: {'line': 'rw43', 'zone': 'zone01', 'station': '-1', 'asset': '-1', 'tag_name': 'Zone01_Shift_1', 
#                   'tag_value': '1', 'tag_datetime':'2020-05-28T11:31:05.89'} 
key_conversion={'root.LaneID':'line','root.ZoneID':'zone','root.TagID':'tag_name','root.StationID':'station',
                        'root.EquipmentID':'asset','root.TagValue':'tag_value','root.TagDateTime':'tag_datetime'}

msg_count=0
consumer = KafkaConsumer(topic_name, bootstrap_servers=b_server,auto_offset_reset='earliest',
                          enable_auto_commit=True, value_deserializer=lambda x: loads(x.decode('utf-8')))
for msg in consumer:
    msg_count=msg_count+1
    msg = msg.value
    
    print('\n')
    sys.stdout.write('%d'%msg_count)
    
    #print('{}'.format(msg))
  
    #parse tag from msg
    dic_msg=msg[0]#type(msg)=list, type(msg[0])=dic\n",
    res=json.loads(dic_msg['FeedData'])#type(dic_msg['FeedData'])=str, type(dic_msg['FeedData'][0])=type(res)=list\n",
    new_dic=res[0]#type(new_dic)=dic\n",
    name_list=new_dic['NAMES']#type(new_dic['NAMES'])=list\n",
    value_list=new_dic['VALUES'][0]#type(new_dic['Values'])=list of list\n",
  
    #make one dic from one parsed msg => tag
    plc_dic={}
    for name,value in zip(name_list,value_list):
        #print(name,':',value)
        plc_dic[key_conversion[name]]=value
    
    
    #read/write one tag/msg on redis
    r_val=json.dumps(plc_dic)
    r.set('plc_tag_msg',r_val)
    r_had=r.get('plc_tag_msg')
    plc_tag=json.loads(r_had)
    print('from redis read: ',plc_tag)
    
    
    
    
    #print(plc_dic)
    print('tag_name: ',plc_dic['tag_name'])
    print('tag_value: ',plc_dic['tag_value'])
    
  
    if msg_count==15:
        break
    #break

ModuleNotFoundError: ignored

In [ ]:
consumer = KafkaConsumer(topic_name, bootstrap_servers=b_server,auto_offset_reset='earliest',
                          group_id='communication_module',
                          enable_auto_commit=False, 
                          max_poll_records=2, max_poll_interval_ms= 300000, 
                          value_deserializer=lambda x: loads(x.decode('utf-8')))


NameError: ignored

In [ ]:
import json 
from kafka import KafkaConsumer
import redis
import time
import sys

topic_name = 'magnaPlcTagData'
b_server=['10.0.1.96:9092']


#redis host
redis_host="192.168.2.11"
redis_port = 6379
redis_password = ""
redis.Redis(host="192.168.2.11", port=6379, db=0)

#key conversion: {'line': 'rw43', 'zone': 'zone01', 'station': '-1', 'asset': '-1', 'tag_name': 'Zone01_Shift_1', 
#                   'tag_value': '1', 'tag_datetime':'2020-05-28T11:31:05.89'} 
key_conversion={'root.LaneID':'line','root.ZoneID':'zone','root.TagID':'tag_name','root.StationID':'station',
                        'root.EquipmentID':'asset','root.TagValue':'tag_value','root.TagDateTime':'tag_datetime'}

msg_count=0
consumer = KafkaConsumer(topic_name, bootstrap_servers=b_server,auto_offset_reset='earliest',
                          enable_auto_commit=True, value_deserializer=lambda x: loads(x.decode('utf-8')))
for msg in consumer:
    msg_count=msg_count+1
    msg = msg.value
    
    print('\n')
    sys.stdout.write('%d'%msg_count)
    
    #print('{}'.format(msg))
  
    #parse tag from msg
    dic_msg=msg[0]#type(msg)=list, type(msg[0])=dic\n",
    res=json.loads(dic_msg['FeedData'])#type(dic_msg['FeedData'])=str, type(dic_msg['FeedData'][0])=type(res)=list\n",
    new_dic=res[0]#type(new_dic)=dic\n",
    name_list=new_dic['NAMES']#type(new_dic['NAMES'])=list\n",
    value_list=new_dic['VALUES'][0]#type(new_dic['Values'])=list of list\n",
  
    #make one dic from one parsed msg => tag
    plc_dic={}
    for name,value in zip(name_list,value_list):
        #print(name,':',value)
        plc_dic[key_conversion[name]]=value
    
    #print(plc_dic)
    print('tag_name: ',plc_dic['tag_name'])
    print('tag_value: ',plc_dic['tag_value'])
  
    if msg_count==15:
        break
    #break

ModuleNotFoundError: ignored

In [ ]:
import json
from kafka import KafkaConsumer
import redis
import time
import sys
    
# kafka host (broker)
topic_name = 'magnaPlcTagData'
b_server=['10.0.1.96:9092']

#redis host
redis_host="192.168.2.11"
redis_port = 6379
redis_password = ""
redis.Redis(host="192.168.2.11", port=6379, db=0)

#key conversion: {'line': 'rw43', 'zone': 'zone01', 'station': '-1', 'asset': '-1', 'tag_name': 'Zone01_Shift_1', 
#                   'tag_value': '1', 'tag_datetime':'2020-05-28T11:31:05.89'} 
key_conversion={'root.LaneID':'line','root.ZoneID':'zone','root.TagID':'tag_name','root.StationID':'station',
                        'root.EquipmentID':'asset','root.TagValue':'tag_value','root.TagDateTime':'tag_datetime'}

msg_count=0
consumer = KafkaConsumer(topic_name, bootstrap_servers=b_server,auto_offset_reset='earliest',
                           enable_auto_commit=True, value_deserializer=lambda x: loads(x.decode('utf-8')))
    
for msg in consumer:
  msg = msg.value
  msg_count=msg_count+1
  sys.stdout.write('\\r%d'%msg_count) 
  print('{}'.format(msg))
  
  #parse tag from msg
  dic_msg=msg[0]#type(msg)=list, type(msg[0])=dic\n",
  res=json.loads(dic_msg['FeedData'])#type(dic_msg['FeedData'])=str, type(dic_msg['FeedData'][0])=type(res)=list\n",
  new_dic=res[0]#type(new_dic)=dic\n",
  name_list=new_dic['NAMES']#type(new_dic['NAMES'])=list\n",
  value_list=new_dic['VALUES'][0]#type(new_dic['Values'])=list of list\n",
  
  #make one dic from one parsed msg => tag
  plc_dic={}
  for name,value in zip(name_list,value_list):
      #print(name,':',value)\n",
      plc_dic[key_conversion[name]]=value
      #print(plc_dic)\n",
      print(plc_dic['tag_name'])
    
      if msg_count==1:
        first_tag=plc_dic['tag_name']
        if plc_dic['tag_name']==first_tag:
          print ('tag repeats')
          print('this_tag',plc_dic['tag_name'])
          print('msg_count',msg_count)
          break
          sys.stdout.flush()
        time.sleep(0.2)
        if msg_count==15:
            break\n",
        #break\n",
    

SyntaxError: ignored

In [ ]:
#new 2
import json 
from kafka import KafkaConsumer
import redis
import time
import sys

topic_name = 'magnaPlcTagData'
b_server=['10.0.1.96:9092']


#redis host
redis_host="192.168.2.11"
redis_port = 6379
redis_password = ""
r=redis.Redis(host="192.168.2.11", port=6379, db=0)

#key conversion: {'line': 'rw43', 'zone': 'zone01', 'station': '-1', 'asset': '-1', 'tag_name': 'Zone01_Shift_1', 
#                   'tag_value': '1', 'tag_datetime':'2020-05-28T11:31:05.89'} 
key_conversion={'root.LaneID':'line','root.ZoneID':'zone','root.TagID':'tag_name','root.StationID':'station',
                
                        'root.EquipmentID':'asset','root.TagValue':'tag_value','root.TagDateTime':'tag_datetime'}

msg_count=0
plc_tag_hash={}#dic of dic
plc_tag_msg={}
consumer = KafkaConsumer(topic_name, bootstrap_servers=b_server,auto_offset_reset='earliest',
                          enable_auto_commit=True, value_deserializer=lambda x: loads(x.decode('utf-8')))
for msg in consumer:
    msg_count=msg_count+1
    msg = msg.value
    
    #print('{0}\r'.format(msg_count)),
    #print(msg_count,end=' ',flush=True)
    #sys.stdout.write('%d'%msg_count)
    #print(msg_count)
    #sys.stdout.flush()
    #time.sleep(0.1)
    
    #print('{}'.format(msg))
  
    #parse tag from msg
    dic_msg=msg[0]#type(msg)=list, type(msg[0])=dic\n",
    res=json.loads(dic_msg['FeedData'])#type(dic_msg['FeedData'])=str, type(dic_msg['FeedData'][0])=type(res)=list\n",
    new_dic=res[0]#type(new_dic)=dic\n",
    name_list=new_dic['NAMES']#type(new_dic['NAMES'])=list\n",
    value_list=new_dic['VALUES'][0]#type(new_dic['Values'])=list of list\n",
  
    #make one dic from one parsed msg => tag
    plc_dic={}
    for name,value in zip(name_list,value_list):
        #print(name,':',value)
        if value is None:
            value=str(-1)
        plc_dic[key_conversion[name]]=str(value).lower()
        
    #print(plc_dic)
    
    #make primary key for the tag
    pk_tag=plc_dic['line']+'.'+plc_dic['zone']+'.'+plc_dic['station']+'.'+plc_dic['asset']+'.'+plc_dic['tag_name']
    pk_tag=pk_tag.replace('.-1','')
    #print(pk_tag)
    
    plc_tag_msg[pk_tag]=plc_dic
    
    
    #redis write    
    r_val=json.dumps(plc_tag_msg)
    #r.set('plc_tag_hash',r_val)#UNCOMMENT
    
    #redis read
    '''
    r_had=r.get('plc_tag_hash')
    plc_tag=json.loads(r_had)
    #print('from redis read: ',plc_tag)
    #'''    
    
    
    
    #print(plc_dic)
    #print('tag_name: ',plc_dic['tag_name'])
    #print('tag_value: ',plc_dic['tag_value'])
    
    if msg_count%100==0:
        print(msg_count,end=' ')
    if msg_count==30000:
        print('done')
        break
    #break

In [ ]:
from json import loads
from kafka import KafkaConsumer

topic_name = 'magnaPlcTagData'
b_server=['10.0.1.96:9092']

consumer = KafkaConsumer(topic_name, bootstrap_servers=b_server,
                          enable_auto_commit=True, value_deserializer=lambda x: loads(x.decode('utf-8')))
for message in consumer:
  message = message.value
  print('{}'.format(message))
  break

ModuleNotFoundError: ignored

In [ ]:
!pip install kafka-python
from time import sleep
from json import dumps
from kafka import KafkaProducer
from kafka import KafkaConsumer

In [ ]:
from json import loads
consumer = KafkaConsumer(
    'spring_test',
     bootstrap_servers=['localhost:9092'],
     auto_offset_reset='earliest',
     enable_auto_commit=True,
     group_id='my-group',
     value_deserializer=lambda x: loads(x.decode('utf-8')));

for message in consumer:
  message = message.value;
  print('{}'.format(message))

In [ ]:
topic_name = 'magnaPlcTagData'
consumer = KafkaConsumer(topic_name, auto_offset_reset='earliest', 
                         bootstrap_servers=['10.0.1.96:9092'], api_version=(0, 10), consumer_timeout_ms=1000000)
for msg in consumer:
  print(msg)

In [ ]:
from kafka import KafkaProducer
producer = KafkaProducer(bootstrap_servers='10.0.1.96:9092')
producer.send('sample', b'Hello, World!')
producer.send('sample', key=b'message-two', value=b'This is Kafka-Python')

from kafka import KafkaConsumer
consumer = KafkaConsumer('sample')
for message in consumer:
    print (message)

In [ ]:
if __name__ == '__main__':
    print('Running Consumer..')
    parsed_records = []
    topic_name = 'raw_recipes'
    parsed_topic_name = 'parsed_recipes'

    consumer = KafkaConsumer(topic_name, auto_offset_reset='earliest',
                             bootstrap_servers=['localhost:9092'], api_version=(0, 10), consumer_timeout_ms=1000)
    for msg in consumer:
        html = msg.value
        result = parse(html)
        parsed_records.append(result)
    consumer.close()
    sleep(5)

    if len(parsed_records) > 0:
        print('Publishing records..')
        producer = connect_kafka_producer()
        for rec in parsed_records:
            publish_message(producer, parsed_topic_name, 'parsed', rec)

Running Consumer..


In [ ]:
#IP: 10.0.1.96
#Port: 9092
#Topic name: magnaPlcTagData
from time import sleep
from json import dumps
from kafka import KafkaProducer
from kafka import KafkaConsumer


if __name__ == '__main__':
    print('Running Consumer..')
    parsed_records = []
    topic_name = 'magnaPlcTagData'
    parsed_topic_name = 'parsed_recipes'

    #consumer = KafkaConsumer(topic_name, auto_offset_reset='earliest', bootstrap_servers=['localhost:9092'], api_version=(0, 10), consumer_timeout_ms=1000)
    consumer = KafkaConsumer(topic_name, auto_offset_reset='earliest', bootstrap_servers=['10.0.1.96:9092'], api_version=(0, 10), consumer_timeout_ms=1000)
    for msg in consumer:
        html = msg.value
        result = parse(html)
        parsed_records.append(result)
    consumer.close()
    sleep(5)

    if len(parsed_records) > 0:
        print('Publishing records..')
        producer = connect_kafka_producer()
        for rec in parsed_records:
            publish_message(producer, parsed_topic_name, 'parsed', rec)

Running Consumer..


In [ ]:
topic_name = 'magnaPlcTagData'
consumer = KafkaConsumer(topic_name, auto_offset_reset='earliest', 
                         bootstrap_servers=['10.0.1.96:9092'], api_version=(0, 10), consumer_timeout_ms=1000)
for msg in consumer:
  print(msg.key)

In [ ]:
from kafka import KafkaConsumer

# To consume latest messages and auto-commit offsets
consumer = KafkaConsumer('magnaPlcTagData', bootstrap_servers=['10.0.1.96:9092'],api_version=(0, 10, 1))
for message in consumer:
    # message value and key are raw bytes -- decode if necessary!
    # e.g., for unicode: `message.value.decode('utf-8')`
    #print ("%s:%d:%d: key=%s value=%s" % (message.topic, message.partition,
    #                                     message.offset, message.key,
    #                                      message.value))
    print(msg.key)

In [ ]:
#pycharm code
#import kafka
from kafka import KafkaConsumer
import pandas
topic_name = 'magnaPlcTagData'
consumer = KafkaConsumer(topic_name, auto_offset_reset='earliest',
                         bootstrap_servers=['10.0.1.96:9092'], api_version=(0, 10), consumer_timeout_ms=10)

#consumer = KafkaConsumer(topic_name, bootstrap_servers=['10.0.1.96:9092'])
for msg in consumer:
  print(msg)

# communication_module

In [ ]:
#Mqtt pulls from redis keys

#!pip3 install paho-mqtt
import paho.mqtt.client as mqtt #import the client1
broker_address="localhost"#"192.168.1.184" 
#broker_address="iot.eclipse.org" #use external broker
client = mqtt.Client("P1") #create new instance
client.connect(broker_address) #connect to broker
client.publish("house/main-light","OFF")#publish

In [ ]:
import requests

#url = "https://plc-iot.c2m.net/api/v1/data/process"
#url = "https://nam04.safelinks.protection.outlook.com/?url=https%3A%2F%2Fhelio-ent.c2m.net%2Flogin.aspx&data=02%7C01%7Cayeshas%40okstate.edu%7Ce33dcfb71016446bad6608d7f6b3bb95%7C2a69c91de8494e34a230cdf8b27e1964%7C0%7C1%7C637249124117860660&sdata=%2F6sf3qNRTsDbmWgWdUDKsS%2FrOy6vJ%2FiHgA1ocMWNMbo%3D&reserved=0"
#url = "https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=XipmK93J5vGOb8hvZ/S8rge856C/TG6ovA7DCelU@/Y=&parameterType=LISTDATA"
url = "https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=XipmK93J5vGOb8hvZ/S8rge856C/TG6ovA7DCelU@/Y=&parameterType=LISTDATA"
#headers = {"Authorization": "CSWQpToX4KLCwODDd3QCmBzak8/DYJ" }
response = requests.get(url)#,headers=headers)
print(response.text)


# Standard API format to request data from helio server

In [ ]:
import requests
token_string = ""
url=""
headers= {"Authorization": token_string}
response=requests.get(url,headers=headers)

# Requesting data from PLC feed of helio server

feedID= XipmK93J5vGOb8hvZ/S8rge856C/TG6ovA7DCelU@/Y=<br>
Given Feed ID: iGfXcckbZrSCd8G4xjek1TV9t@ZbzdAP4lU0/bUeyQQ= 

In [ ]:
import requests
url = "https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=XipmK93J5vGOb8hvZ/S8rge856C/TG6ovA7DCelU@/Y=&parameterType=LISTDATA"

#url="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=iGfXcckbZrSCd8G4xjek1TV9t@ZbzdAP4lU0/bUeyQQ=&parameterType=LISTDATA"
#headers = {"Authorization": "CSWQpToX4KLCwODDd3QCmBzak8/DYJ" }
response = requests.get(url)#,headers=headers)
print(response.text)


# Print XML plc tags as string

In [ ]:
import xml.etree.ElementTree as ET

response_text=ET.fromstring(response.text)#prints wordpad with xd 
response_xmlstr = ET.tostring(response_text, encoding='utf8', method='xml')#prints 1 line but has all the info
xml_str = response_xmlstr.decode()#prints wordpad without xd

#print(type(response_text))#<class 'xml.etree.ElementTree.Element'>
#print(type(response_xmlstr))# <class 'bytes'>
#print(type(xml_str))# <class 'str'>
print(xml_str)# <class 'str'>


# Traversing the xml tree of plc tags

In [ ]:
for node in response_text.iter():
  print (node.tag, node.attrib, node.text)
  #print(node)#Elements, such as <Element 'root.EquipmentID' at 0x7f05be878868>, <Element 'root.TagDateTime' at 0x7f05be878598>,<Element 'Table1' at 0x7f05be8785e8>
  #print(node.text)
  #if node.text=='RW43.PLC01.Zone1.ST045.Equip018.CycleTime':
    #print(node.tag)

In [ ]:
#install and import redis
!pip install redis
!pip install t_log
import redis

In [ ]:
#how to setup connection with redis, where do I get password?
# https://opensource.com/article/18/4/how-build-hello-redis-with-python
#next task would be write to redis
#then next task would be read back from redis by reconnecting
import redis

redis_host = "localhost"
redis_port = 6379
redis_password = ""
'''
r = pyredis.Redis(host='localhost', port=redis_port, password=redis_password)
r.set("msg:hello", "Hello Redis!!!")
msg = r.get("msg:hello")
print(msg)
#'''

res = redis.Redis(host='47.99.94.49', port=6379, db=0)
res.set("name","Ayesha")
print(res.get('name'))


In [ ]:
!pip install redis-server
!redis-cli

In [ ]:
!pip install redis

In [ ]:
import redis

# step 2: define our connection information for Redis
# Replaces with your configuration information
redis_host = "192.168.2.11"#"localhost"
redis_port = 6379
redis_password = ""


def hello_redis():
    """Example Hello Redis Program"""
   
    # step 3: create the Redis Connection object
    try:
   
        # The decode_repsonses flag here directs the client to convert the responses from Redis into Python strings
        # using the default encoding utf-8.  This is client specific.
        r = redis.StrictRedis(host=redis_host, port=redis_port, password=redis_password, decode_responses=True)
   
        # step 4: Set the hello message in Redis
        r.set("msg:hello", "Hello Redis!!!")

        # step 5: Retrieve the hello message from Redis
        msg = r.get("msg:hello")
        print(msg)        
   
    except Exception as e:
        print(e)



hello_redis()

Error 110 connecting to 192.168.2.11:6379. Connection timed out.


In [ ]:
import redis


#res = redis.Redis(host='192.168.2.11', port=6379, db=0)
#res = redis.Redis(host="localhost", port=6379,db=0)#password="")
#r = redis.Redis(host='localhost', port=6379, decode_responses=True)
#r = redis.Redis(host='192.168.2.11', port=6379, decode_responses=True)
rC = redis.Redis(host='192.168.2.11', port=6379, db=0)
#rC = redis.StrictRedis(host='localhost',port=6379,db=0)
rC.set("name","Ayesha")
print(rC.get('name'))


In [ ]:
import the Redis client
import redis


# Create a redis client
redisClient = redis.StrictRedis(host='localhost', port=6379, db=0)

# Add key value pairs to the Redis hash
hashName = "Dessert"
redisClient.hset(hashName, 'line', "Cheesecake")
redisClient.hset(hashName, 'zone', "zone01")
redisClient.hset(hashName, 'station', "-1")
redisClient.hset(hashName, 'asset', "-1")
redisClient.hset(hashName, 'tag_name', "Zone01_Shift_1")
redisClient.hset(hashName, 'tag_value', "1")

# Print the hash
print(redisClient.hgetall(hashName))

# Remove a key
redisClient.hdel(hashName, 1)
 

# Print the hash after removing a key
print(redisClient.hgetall(hashName))

{'line': 'rw43',
 'zone': 'zone01',
 'station': '-1',
 'asset': '-1',
 'tag_name': 'Zone01_Shift_1',
 'tag_value': '1'
} 



In [ ]:
import pickle
import redis

r = redis.StrictRedis('localhost')
mydict = {1:2,2:3,3:4}
p_mydict = pickle.dumps(mydict)
r.set('mydict',p_mydict)

read_dict = r.get('mydict')
yourdict = pickle.loads(read_dict)

In [ ]:
!pip install redis

In [ ]:
import redis
r = redis.Redis(host="192.168.2.11", port=6379, db=0)
x = r.get("analytics_results")
print(x)

<br>

# <font color='blue'> Workflow feeds </font>

# Requesting data from Workflow feed of helio server


In [ ]:
import requests


#links for 5 workflow feeds
url_mShift = "https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=moU39Gbxtz9IhQO0gUy@EQAKbJcR374vb@IVb6kWwH0=&parameterType=LISTDATA"# 1 record
url_mJob="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=GIcqmjbvPvCFX3QNIsJbTNa99MyIUuvAFUyKPTBcs9g=&parameterType=LISTDATA" #has zero records
url_mStation="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=GN3jGNVF03thDKOMppdZf/hjLH5vss2SgBlEkNaZvtE=&parameterType=LISTDATA" #20 records
url_mZone="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=LYZEOIUPmsEILn4DoGwhlGw0VaYZ6Yh8@bEKrGOlN6A=&parameterType=LISTDATA" # 2 records
url_mLine="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=XWPuVgYIhUVEZXRsr6gAFq2BPeTCdlLJxRBpvg07WLM=&parameterType=LISTDATA" # 1 record



#get response
url=url_mShift
response = requests.get(url)#,headers=headers)


#print response
print(response.text)



# Print workflow feeds as string

In [3]:
import xml.etree.ElementTree as ET

response_text=ET.fromstring(response.text)#prints wordpad with xd 
response_xmlstr = ET.tostring(response_text, encoding='utf8', method='xml')#prints 1 line but has all the info
xml_str = response_xmlstr.decode()#prints wordpad without xd

#print(type(response_text))#<class 'xml.etree.ElementTree.Element'>
#print(type(response_xmlstr))# <class 'bytes'>
#print(type(xml_str))# <class 'str'>
print(xml_str)# <class 'str'>


<?xml version='1.0' encoding='utf8'?>
<root><response fetchedrecords="3" startrecord="0" status="SUCCESS" totalrecords="3"><NewDataSet>
  <Table1>
    <c2mdatetime>2020-06-30T15:06:05.144Z</c2mdatetime>
    <TRNSCTNID>11856</TRNSCTNID>
    <CRTDON>2020-06-25T03:08:44.000Z</CRTDON>
    <MODFON>2020-06-29T14:01:11.000Z</MODFON>
    <CRTDBY>3871</CRTDBY>
    <MODFBY>3871</MODFBY>
    <fmsscheduleid>1JTE</fmsscheduleid>
    <fmsschedulename>Shift 1</fmsschedulename>
    <fmsschedulelinelink>RW43</fmsschedulelinelink>
    <fmsscheduledays />
    <fmsschedulestarttime>12:00 Am</fmsschedulestarttime>
    <fmsscheduleendtime>08:00 Am</fmsscheduleendtime>
    <fmsschedulebreakstart>02:00 Am</fmsschedulebreakstart>
    <fmsschedulebreakend>02:15 Am</fmsschedulebreakend>
    <fmsschedulelunchstart>04:00 Am</fmsschedulelunchstart>
    <fmsschedulelunchend>05:00 Am</fmsschedulelunchend>
    <fmsscheduledaysnew>Monday!Tuesday!Wednesday!Thursday!Friday</fmsscheduledaysnew>
    <StageName>Status</Stag

# Parse workflow feed which is an xml tree & put in a dic

In [ ]:
for node in response_text.iter():
  #print (node.tag, node.attrib, node.text)
  
  print(node.tag)#Elements, such as <Element 'root.EquipmentID' at 0x7f05be878868>, <Element 'root.TagDateTime' at 0x7f05be878598>,<Element 'Table1' at 0x7f05be8785e8> KEY
  print(node.attrib)
  print(node.text) #should go to dic value? make that sure VALUE
  #print(node.text)
  #if node.text=='RW43.PLC01.Zone1.ST045.Equip018.CycleTime':
    #print(node.tag)

root
{}
None
response
{'status': 'SUCCESS', 'startrecord': '0', 'fetchedrecords': '3', 'totalrecords': '3'}
None
NewDataSet
{}

  
Table1
{}

    
c2mdatetime
{}
2020-06-29T14:41:50.951Z
TRNSCTNID
{}
11857
CRTDON
{}
2020-06-29T13:58:23.000Z
MODFON
{}
2020-06-29T13:58:23.000Z
CRTDBY
{}
3871
MODFBY
{}
3871
fmsscheduleid
{}
1QRL
fmsschedulename
{}
Shift 2
fmsschedulelinelink
{}
RW43
fmsscheduledays
{}
None
fmsschedulestarttime
{}
08:00 Am
fmsscheduleendtime
{}
04:00 Pm
fmsschedulebreakstart
{}
10:00 Am
fmsschedulebreakend
{}
10:15 Am
fmsschedulelunchstart
{}
12:00 Pm
fmsschedulelunchend
{}
01:00 Pm
fmsscheduledaysnew
{}
Monday!Tuesday!Wednesday!Thursday!Friday
StageName
{}
Status
StateName
{}
Active
username
{}
lmv_admin@yopmail.com
companyname
{}
MagnaLMVCompany4
groupname
{}
MagnaLMVCompany4
Table1
{}

    
c2mdatetime
{}
2020-06-29T14:41:50.951Z
TRNSCTNID
{}
11858
CRTDON
{}
2020-06-29T14:02:33.000Z
MODFON
{}
2020-06-29T14:02:33.000Z
CRTDBY
{}
3871
MODFBY
{}
3871
fmsscheduleid
{}
15PN
f